In [79]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
import numpy as np
import pandas as pd
import os
from tqdm import tqdm 
# import matplotlib.pyplot as plt

In [2]:
## tokenize the text..
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)

In [3]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [5]:
# setup env
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
## load the embeddings (glove twitter and get the metrics from bi-LSTM) 

In [7]:
dir_path = "/home/samarthgoal/language_vision/input/reddit/"

In [8]:
df_rage = pd.read_csv(os.path.join(dir_path,'processed_rage.csv'))
df_happy =  pd.read_csv(os.path.join(dir_path,'processed_happy.csv'))
df_gore =  pd.read_csv(os.path.join(dir_path,'processed_gore.csv'))
df_creepy =  pd.read_csv(os.path.join(dir_path,'processed_creepy.csv'))

In [10]:
# create a random balances dataset of all of the categories
length = np.min([len(df_rage),len(df_happy),len(df_creepy),len(df_gore)])

In [11]:
df_final = pd.concat([df_rage[:length], df_happy[:length], df_gore[:length], df_creepy[:length]], ignore_index=True)

In [12]:
# create a random balances dataset of all of the categories
Y_new = df_final['subreddit']

In [13]:
# encode the output
Y_new = le.fit_transform(Y_new) 

In [14]:
token_list = (df_final['title'].apply(get_tokens))

In [ ]:
## join the filtered tokens back again to the keras tokenizer which would give vocalb words etc
from keras.preprocessing.text import Tokenizer
sentences = [' '.join(tokens) for tokens in token_list]

In [45]:
max_len

214

In [15]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1
vocab_size

4932

In [51]:
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(sentences)
# pad documents to a max length of 4 words
max_length = max_len
X = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
y = Y_new

In [52]:
## now splitting into test and training data
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train, Y_test =  train_test_split(X, y,test_size =0.20,random_state= 4 )

In [47]:
embedding_path = "/home/samarthgoal/embeddings/glove.twitter.27B.100d.txt" ## change

In [48]:
# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    file = open(embedding_path, "r")
    if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
            key = line.split(' ',1)[0] # the first word is the key
            value = np.array([float(val) for val in line.split(' ')[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")

In [24]:
w2v = get_word2vec(embedding_path)

In [25]:
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = w2v.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [72]:
model = Sequential()
e =  Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add( Bidirectional( LSTM(100,dropout=0.20),merge_mode='concat'))
model.add( Dense(4, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [73]:
hist = model.fit(X_train, Y_train,validation_split=0.25, nb_epoch = 10, verbose = 2)

/home/samarthgoal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 1476 samples, validate on 492 samples
Epoch 1/10
 - 49s - loss: 1.2330 - acc: 0.4661 - val_loss: 0.9811 - val_acc: 0.6504
Epoch 2/10
 - 37s - loss: 0.8907 - acc: 0.6592 - val_loss: 0.8051 - val_acc: 0.7012
Epoch 3/10
 - 37s - loss: 0.7637 - acc: 0.7134 - val_loss: 0.8009 - val_acc: 0.7012
Epoch 4/10
 - 37s - loss: 0.7118 - acc: 0.7344 - val_loss: 0.7844 - val_acc: 0.7256
Epoch 5/10
 - 37s - loss: 0.6669 - acc: 0.7425 - val_loss: 0.7323 - val_acc: 0.7297
Epoch 6/10
 - 37s - loss: 0.6125 - acc: 0.7649 - val_loss: 0.7572 - val_acc: 0.7276
Epoch 7/10
 - 37s - loss: 0.5817 - acc: 0.7778 - val_loss: 0.7439 - val_acc: 0.7276
Epoch 8/10
 - 37s - loss: 0.5387 - acc: 0.8008 - val_loss: 0.7417 - val_acc: 0.7398
Epoch 9/10
 - 37s - loss: 0.5331 - acc: 0.7974 - val_loss: 0.7168 - val_acc: 0.7256
Epoch 10/10
 - 37s - loss: 0.5001 - acc: 0.8103 - val_loss: 0.7458 - val_acc: 0.7236


plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [68]:
# evaluate the model <100 LSTMlayer>
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 73.780488


In [95]:
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.69      0.70       129
           1       0.62      0.68      0.65       106
           2       0.77      0.85      0.81       127
           3       0.79      0.66      0.72       130

   micro avg       0.72      0.72      0.72       492
   macro avg       0.72      0.72      0.72       492
weighted avg       0.73      0.72      0.72       492
 



In [65]:
# evaluate the model <>
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 73.983740
